# Submitting (uploading) data to the JCOIN Data Commons

To upload (and update) a core measure (or other package or file for that matter) to the JDC, one can use the `submit_package_to_jdc` and `map_to_sheepdog` functions (in this case, we use `map_core_measures_to_sheepdog` which runs a set of transforms to go from a validated core measure data package to a sheepdog submission and then uses `map_to_sheepdog` in the `jdc_utils.utils.gen3` module). Sheepdog maps a subset of variables for quicker discovery on the JDC.


## Create a new file record (does have any old version- entirely new file)

In [1]:
from jdc_utils.submission import submit_package_to_jdc,map_core_measures_to_sheepdog
from frictionless import Package
import os

In [7]:
from gen3.auth import Gen3Auth
from gen3.submission import Gen3Submission
from gen3.index import Gen3Index
from gen3.file import Gen3File
index = Gen3Index(Gen3Auth(refresh_file="credentials.json"))
sub = Gen3Submission(Gen3Auth(refresh_file="credentials.json"))
files = Gen3File(Gen3Auth(refresh_file="credentials.json"))
sheepdog_rec = sub.export_node("JCOIN","TEST","reference_file",fileformat="json")
index_rec = index.get_record(sheepdog_rec["data"][-1]["object_id"])
# files.delete_file_locations(index_rec["object_id"])
# sub.delete_record("JCOIN","TEST",sheepdog_rec["data"][-1]["id"])

[2023-04-07 11:27:48,314][WARNING] failed to write token cache file: C:\Users\kranz-michael/.cache/gen3/token_cache_243db9dca6c478c2a6ee5ce88b9e5916
[2023-04-07 11:27:48,315][WARNING] [WinError 183] Cannot create a file when that file already exists: 'C:\\Users\\kranz-michael/.cache/gen3/token_cache_243db9dca6c478c2a6ee5ce88b9e5916.tmp_eraseme_3647_1680884868' -> 'C:\\Users\\kranz-michael/.cache/gen3/token_cache_243db9dca6c478c2a6ee5ce88b9e5916'
[2023-04-07 11:27:48,316][WARNING] backoff: call gen3.auth._write_to_file(<gen3.auth.Gen3Auth object at 0x0000020467BE2B30>, C:\Users\kranz-michael/.cache/gen3/token_cache_243db9dca6c478c2a6ee5ce88b9e5916, eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsImtpZCI6ImZlbmNlX2tleV8yMDIwLTAxLTAyVDA2OjM1OjA4WiJ9.eyJwdXIiOiJhY2Nlc3MiLCJpc3MiOiJodHRwczovL2pjb2luLmRhdGFjb21tb25zLmlvL3VzZXIiLCJhdWQiOlsiaHR0cHM6Ly9qY29pbi5kYXRhY29tbW9ucy5pby91c2VyIiwiZ29vZ2xlX3NlcnZpY2VfYWNjb3VudCIsInVzZXIiLCJvcGVuaWQiLCJmZW5jZSIsImRhdGEiLCJnb29nbGVfY3JlZGVudGlhbHMiLCJhZG1pbiIsImdvb2d

In [ ]:
# create new record - do this only one  time!
# NOTE: no file guid because the record has not been created yet
# NOTE: if want to test with new file records, delete the ones added by
 # uncommenting the deletion methods in above cell
gen3_file = submit_package_to_jdc(
    package_path=zipped_package_path,
    commons_project=commons_project,
    sheepdog_file_submitter_id='core-measures-TEST',
    sheepdog_data_type="Interview",
    sheepdog_data_category="Core Measures",
    sheepdog_data_format="ZIP",
    submission_type="create",
)

Save the metadata to a file for later reference when updating file

In [30]:
#TODO

## Map variables to sheepdog 
This uses the specific `map_core_measures_to_sheepdog` which calls the 
more general `map_to_sheepdog` function

In [ ]:
package_path = zipped_package_path.replace(".zip","")+"/data-package.json"
core_measures = Package(package_path)
baseline_df = core_measures.get_resource("baseline").to_pandas()
baseline_df["role_in_project"] = "Client"
timepoints_df = core_measures.get_resource("timepoints").to_pandas()
last_node_output = map_core_measures_to_sheepdog(
    baseline_df=baseline_df,
    timepoints_df=timepoints_df,
    program="JCOIN",
    project="TEST",
    credentials_path="credentials.json"
)

## Update file record

To simulate an update: Change something in package, rezip, and run upload/sheepdog mapping

In [12]:
from jdc_utils.submission import CoreMeasures,map_core_measures_to_sheepdog,submit_package_to_jdc
from pathlib import Path
import os
from gen3.auth import Gen3Auth
from gen3.submission import Gen3Submission

In [2]:
# make slightly updated zipped file
core_measures = CoreMeasures("../data/external_workflow_slight_update",is_core_measures=True)
core_measures.write(outdir="../data/core-measures-slight-update-test")
core_measures.zip()

c:\Users\kranz-michael\projects\jdc-utilities\notebooks
C:\Users\kranz-michael\projects\jdc-utilities\data\external_workflow_slight_update


In [3]:
commons_project = "TEST"
sub = Gen3Submission(Gen3Auth(refresh_file="credentials.json"))
sheepdog_rec = sub.export_node("JCOIN","TEST","reference_file",fileformat="json")
file_guid = sheepdog_rec["data"][-1]["object_id"] # assumes you just ran the "create" section of notebook

[2023-04-07 11:48:12,785][WARNING] failed to write token cache file: C:\Users\kranz-michael/.cache/gen3/token_cache_243db9dca6c478c2a6ee5ce88b9e5916
[2023-04-07 11:48:12,786][WARNING] [WinError 183] Cannot create a file when that file already exists: 'C:\\Users\\kranz-michael/.cache/gen3/token_cache_243db9dca6c478c2a6ee5ce88b9e5916.tmp_eraseme_92758_1680886092' -> 'C:\\Users\\kranz-michael/.cache/gen3/token_cache_243db9dca6c478c2a6ee5ce88b9e5916'
[2023-04-07 11:48:12,790][WARNING] backoff: call gen3.auth._write_to_file(<gen3.auth.Gen3Auth object at 0x0000028CE13E7D90>, C:\Users\kranz-michael/.cache/gen3/token_cache_243db9dca6c478c2a6ee5ce88b9e5916, eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsImtpZCI6ImZlbmNlX2tleV8yMDIwLTAxLTAyVDA2OjM1OjA4WiJ9.eyJwdXIiOiJhY2Nlc3MiLCJpc3MiOiJodHRwczovL2pjb2luLmRhdGFjb21tb25zLmlvL3VzZXIiLCJhdWQiOlsiaHR0cHM6Ly9qY29pbi5kYXRhY29tbW9ucy5pby91c2VyIiwiZ29vZ2xlX3NlcnZpY2VfYWNjb3VudCIsInVzZXIiLCJvcGVuaWQiLCJmZW5jZSIsImRhdGEiLCJnb29nbGVfY3JlZGVudGlhbHMiLCJhZG1pbiIsImdvb2

In [4]:
print(file_guid)

dg.6VTS/ebb26cad-3acb-4969-8a4c-2869a2e6121f


In [6]:
gen3_file = submit_package_to_jdc(
    package_path="../data/core-measures-slight-update-test.zip",
    commons_project=commons_project,
    file_guid=file_guid,
    sheepdog_file_submitter_id='core-measures-test',
    sheepdog_data_type="Interview",
    sheepdog_data_category="Core Measures",
    sheepdog_data_format="ZIP",
    submission_type="update",
)

[2023-04-07 12:22:22,146][WARNING] failed to write token cache file: C:\Users\kranz-michael/.cache/gen3/token_cache_243db9dca6c478c2a6ee5ce88b9e5916
[2023-04-07 12:22:22,147][WARNING] [WinError 183] Cannot create a file when that file already exists: 'C:\\Users\\kranz-michael/.cache/gen3/token_cache_243db9dca6c478c2a6ee5ce88b9e5916.tmp_eraseme_97699_1680888142' -> 'C:\\Users\\kranz-michael/.cache/gen3/token_cache_243db9dca6c478c2a6ee5ce88b9e5916'
[2023-04-07 12:22:22,148][WARNING] backoff: call gen3.auth._write_to_file(<gen3.auth.Gen3Auth object at 0x0000028CE1E3B4F0>, C:\Users\kranz-michael/.cache/gen3/token_cache_243db9dca6c478c2a6ee5ce88b9e5916, eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsImtpZCI6ImZlbmNlX2tleV8yMDIwLTAxLTAyVDA2OjM1OjA4WiJ9.eyJwdXIiOiJhY2Nlc3MiLCJpc3MiOiJodHRwczovL2pjb2luLmRhdGFjb21tb25zLmlvL3VzZXIiLCJhdWQiOlsiaHR0cHM6Ly9qY29pbi5kYXRhY29tbW9ucy5pby91c2VyIiwiZ29vZ2xlX3NlcnZpY2VfYWNjb3VudCIsInVzZXIiLCJvcGVuaWQiLCJmZW5jZSIsImRhdGEiLCJnb29nbGVfY3JlZGVudGlhbHMiLCJhZG1pbiIsImdvb2

After this step would then map to sheepdog

In [13]:
baseline_df = core_measures.package.get_resource("baseline").to_pandas()
baseline_df["role_in_project"] = "Client" # TODO: add staff
timepoints_df = core_measures.package.get_resource("timepoints").to_pandas()
last_node_output = map_core_measures_to_sheepdog(
    baseline_df=baseline_df,
    timepoints_df=timepoints_df,
    program="JCOIN",
    project="TEST",
    credentials_path="credentials.json"
)

Validating records for participant
Validating records for enrollment
Validating records for demographic_baseline
Validating records for time_point
Deleting records for time_point
[2023-04-07 12:29:08,723][WARNING] failed to write token cache file: C:\Users\kranz-michael/.cache/gen3/token_cache_243db9dca6c478c2a6ee5ce88b9e5916
[2023-04-07 12:29:08,723][WARNING] [WinError 183] Cannot create a file when that file already exists: 'C:\\Users\\kranz-michael/.cache/gen3/token_cache_243db9dca6c478c2a6ee5ce88b9e5916.tmp_eraseme_34386_1680888548' -> 'C:\\Users\\kranz-michael/.cache/gen3/token_cache_243db9dca6c478c2a6ee5ce88b9e5916'
[2023-04-07 12:29:08,724][WARNING] backoff: call gen3.auth._write_to_file(<gen3.auth.Gen3Auth object at 0x0000028CE10A1630>, C:\Users\kranz-michael/.cache/gen3/token_cache_243db9dca6c478c2a6ee5ce88b9e5916, eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsImtpZCI6ImZlbmNlX2tleV8yMDIwLTAxLTAyVDA2OjM1OjA4WiJ9.eyJwdXIiOiJhY2Nlc3MiLCJpc3MiOiJodHRwczovL2pjb2luLmRhdGFjb21tb25zLmlvL3VzZXI